In [1]:
import numpy as np
from create_cat import assign_balrog_zbins, merge_bagpipes, merge_balrog_bagpipes, apply_nzs_weighting, columns_to_drop

# Create Balrog-Bagpipes catalog 

## Load Balrog galaxies and assign redshift bins and n(z)s

In [2]:
balrog_data = assign_balrog_zbins()

Running assign Balrog to zbins

reading pcchat...
Illegal slicing argument for scalar dataspace
just putting None
reading zp...
...success
zeropoint = 30
reading columns...
...success
reading SOMs...
...success
using overlap_weight as stored in CellMap.data

Loaded Balrog entries: 2417437
Unique Balrog sources: 267229

Assigned wide SOM using unsheared fluxes
Assigned wide SOM using sheared_1p fluxes
Assigned wide SOM using sheared_1m fluxes
Assigned wide SOM using sheared_2p fluxes
Assigned wide SOM using sheared_2m fluxes
Completed wide SOM assignment
Completed deep SOM assignment

Assigned tomographic bins
Assigned photometric redshift cell (pzc)


## Load Bagpipes physical properties

In [3]:
bagpipes_data = merge_bagpipes()

Running merge Bagpipes runs

Length of Bagpipes catalog 1: 489046
Length of Bagpipes catalog 2: 358370
Length of Bagpipes catalog 3: 358371
Length of Bagpipes catalog 4: 358370
Length of Bagpipes catalog 5: 358371
Length of Bagpipes extra COSMOS catalog: 57997

Length of merged Bagpipes catalogs: 1979779


## Merge Balrog-Bagpipes

In [4]:
data = merge_balrog_bagpipes(balrog_data, bagpipes_data)

Running merge Balrog with Bagpipes

Length of Balrog single sources matching by coordinates with Bagpipes: 267229
Length of Balrog data matching with Bagpipes: 2417437


## Reweight n(z)s

In [5]:
data['overlap_weight'] = apply_nzs_weighting(data['unsheared/snr'], data['unsheared/T'], data['unsheared/size_ratio'])
data['pzc_weighted'] = data['pzc'] * data['overlap_weight']/np.mean(data['overlap_weight']) # Mean weight is 1

Shear weights adjusted
Response weights adjusted


## Drop some columns

In [6]:
data = data.drop(columns=columns_to_drop)

## Save catalog

In [8]:
file_out = '/global/cfs/cdirs/des/elisa/IA_decisiontree/decisiontree_data/balrog-bagpipes.pkl'
data.to_pickle(file_out)

# Load Balrog-Bagpipes catalog

In [1]:
import numpy as np
import pickle

In [2]:
file = '/global/cfs/cdirs/des/elisa/IA_decisiontree/decisiontree_data/balrog-bagpipes.pkl'
data = pickle.load(open(file, 'rb'), encoding='latin1')
print(f'Length of Balrog-Bagpipes catalog: {len(data)}')

Length of Balrog-Bagpipes catalog: 2417437


In [3]:
data = data.dropna(subset=['stell_best', 'ssfr_best'])
print(f'Length of Balrog-Bagpipes catalog (no nan): {len(data)}')

Length of Balrog-Bagpipes catalog (no nan): 2417396


In [4]:
for i in range(4):
    print(f'Length of Balrog-Bagpipes Bin{i}: {len(data[data["bin"]==i])}')

Length of Balrog-Bagpipes Bin0: 613544
Length of Balrog-Bagpipes Bin1: 608122
Length of Balrog-Bagpipes Bin2: 605458
Length of Balrog-Bagpipes Bin3: 590272
